![Quantinuum_logo](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAIIAAABLCAMAAACLOq%2BzAAACx1BMVEUAAAAAAP%2BAgIBVqqqAgIBmmZmAgKptkpKAgJ9xjo6AgJl0i6JqgJV2iZ1tgJJ3iJlwgJ94h5ZxgJx5hpRzgJl5hp50gJdvhZt1gJVwhZl2gJ12gJtyhJV3gJlzhJx4gJd0g5txgJZ1g5lxgJx1g5hygJp2g5ZzgJl2g5tzgJh3gpp0gJdxgpl0gJtygph1gJpygpd1gJlzgpt2gJhzgpp2gJd0gplygJt0gphygJp1gpdzgJl1gptzgJh1gpp2gZl0gJtygZh0gJpzgZh1gJlzgZp1gJhzgZp1gJh0gZl0gZhygJp0gZhzgJl0gZpzgJh1gZpzgJh1gZl0gJp1gZh0gJp0gJlzgZp0gJhzgZp1gJhzgZl1gJp0gZh1gJp0gZhzgJl0gZpzgJlzgJh1gZlzgJp1gZl1gZh0gJl0gJlzgZl0gJhzgZl1gJp0gZl1gJl0gZh1gJl0gZpzgJl0gZlzgJh0gZlzgJp0gZl0gJl1gJh1gJpzgJl0gJhzgJl0gJpzgJl0gJl1gJl0gJp1gJl0gJl1gJhzgJp0gJlzgJl0gJh0gJl1gJp0gJl1gJl0gJh1gJl0gJpzgJl0gJlzgJh0gJl0gJp0gJl0gJl1gJh0gJl1gJp0gJlzgJl0gJh0gJl0gJp0gJl0gJl0gJh1gJl0gJpzgJh0gJl0gJl0gJl1gJp0gJl1gJl0gJhzgJl0gJl0gJl0gJh0gJl0gJp0gJl0gJl1gJh0gJlzgJp0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJlzgJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJlzgJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl1gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJl0gJn%2F%2F%2F%2Bc0JCdAAAA63RSTlMAAQIDBAUGBwgJCgsMDQ4PEBESExQVFhcYGRocHR4fICEiIyQlJicoKSorLC0uLzAxMjM0NTY3ODk6Ozw9Pj9BQkNERUZHSElKS01OT1BRUlNUVVZXWFpbXF1eX2BhYmNkZWZoaWprbW5wcXJzdHV2d3h5ent8fX5%2FgIGDhYaHiImKjI2Oj5CSk5SVlpeYmZqbnJ2en6ChoqOkpaanqKmqq6ytrq%2Bws7a3ubq7vL2%2BwMHCw8TFxsfIycrLzM3Oz9DR0tPU1dbY2drb3N3e3%2BDh4uPk5ebn6Orr7O3u7%2FHy8%2FT19vf4%2Bfr7%2FP3%2B7Jw5zgAAAAFiS0dE7CG5sxsAAAVfSURBVBgZ7cGNV9X1AcfxNxcwnhRRaVCsKGszI%2BbDcs3KlDKldAmhGywz0ZyJOqLUHrBraVoKJD5E%2BBDaarXS2tZ0jmkqKjlLKR9SA6EFCNzPP7Hv7%2F7QmMm9nM754jkdXi9%2BsNCxL7x9uLb266qKZx%2BI5Ar4yYsn9J2GNyd46F6e%2BfW6RFWmh2404ANdxq6f022uOajL%2BvYPIXSPyN3qTFkY1vUBytW5tyOwLHE%2BjFMg73iwKqpyCCEHFFAhVpXsgfEKzJeBRWO1CDYpiK%2FisCbsU91N%2BDkFU4Q1WVIkqWrX%2BvGTv04Ih96Dp246o47aUrBlpxogR351z8XzHc%2F4j9TBGiz5lVQD82W0Lo7lErfv0EWN8dixTDoICyTV3Mn3heY36YK52LFf%2BgLypH%2FHcVlDT6vddqxI8EnfQLb29qMTg0%2FJ1RSNDQ%2FJiGJEVTydGlQv1%2F3YMEdGGp4BBJAt1xPYsFTGYoLYIL8l2LBZRhVBJDXJ8TI2%2FEmOewhilRwrsGGDHO8QxI1yvIgNa%2BQ3hiCqZCzChmXyq76KwBbLmIsN0%2BQqIbAHZEzGhmFql0dAKTKSsCGiRa622QQSI6kGO7bpgqWhBNAgrcSOabroX7fRuQ8%2Fqx6JHX2bdNH55ddxJZSrg5aNE6Lpdre06v80bnvVW5DfbnZuR6OxZKW6qhxL%2Btaoi97Altub1TWvY81jPnXJeuzJV5esw6LHW9UFa7Ep%2FayCK8Wqaz9QUKux7OFjCqIE28If2aeAiukGtxX%2B47w6VUT3iBg8Mdc1K%2F8S99GjR48ePXr06PHjkzRjycIxHhyR0%2BNwxOT0wxH64G%2F74xrzaDR%2B9%2F4SeDDzghDwZCZh3PdoJH7jhgLXZobhiH84Aph4E36TBgGpmem0S52eDuFLz8vYPwRjiMbj%2BJlycGRLr%2BKq1N964zi0HqjXBTEQockYB7UtGsexImCSYnHMVCJQl4cjwrcAWCaNxy%2F2tE4SskXb74i%2B5omGhhRgpDJwpGgGRtjhszuarsNvd5t2xmJ8XgYMHZ2WdqQuLS1tWBhEaQrGoTb9tTfGVyXAJF2NI0%2BJQF0Bjr5aACz31X%2FaC8cSfXOWDK0LwfjFt7tC4E49gmOEpmFMVW5KSyl%2Be%2F68QJX9gFPluHafwBWj32FUbynUjljg3GrgIf0UR74SgbrncAzQ08CKs7M0DyO56d2t9Ww%2FFYPfUxoOqZqN435lAeGf7fVQ0jIQx97NzNG%2BBKjbiKvyGK4%2BysE4vJYC7Y6H5lJgggbheF6JwJmXcFyvfOCVk6H7ziUAFa23ljdzeg2uFM2CG7QQxxSNA6ZrEiS3lOPYvwHm6EgyjZtx7TqCq69%2Bj3FkNTyp6iSP1gLpugvHMiUCp9bjSFEesPILRqkM7tFrlLbS8Bqu6%2FVHCP36n6EYmxv7QK9jenxKVu6J1sEYB8qAuTpxq68C18fVuOI0FePzYqBANalaDyRrHUbvmuO9gA%2FPxGE8rVFA0VHY4rsr7EBDAkUtHNiJK10ZwHy9lUD0cnmBmWq3FeNQGUaBarUV1%2FYqXP01DeNoMcYi1aoMY5MKI0neoRyMe31%2Fv5FeeS0fhQAlR2Fg8755WgirmnlWo%2FCLHRgKeFZIdY16Ixyu%2BrJy%2BDBHqW8IUF2G4xnpLVzvfYIrXtMxaopxPC%2BVY8T8ReePy1eI38xmX81%2FtedqjNVHgZfV9mUUrGok%2FvjpsXQ0svj90jSMG7zD8Yt%2BZiwwLwu%2FXG82rqzHcMV4R2AU%2FAa%2FGd7JOEImvv7e8mG0u3nJuxuzw3Fk5AP9vd7RwMQCGPQfHayoKClZ6R3FlRJV8MnJ2trG2trZXBH%2FA2RBOeTINIkQAAAAAElFTkSuQmCC)<br>
# Discarding leaky results: automatic leakage error detection with `QuantinuumBackend`

Quantum computers are known to be *noisy*, with a high chance of errors occurring when executing a sequence of operations. These errors can come from a variety of sources and are typically hard to mitigate it. Investigating the source of errors, how they manifest at the Quantum Circuit level, and how to mitigate them is a wide area of research. <br>

A regular experiment using the `QuantinuumBackend` works as follows: circuits are defined using `pytket`, compiled to the device constraints using `QuantinuumBackend.get_compiled_circuit`, sent to the device provider using `QuantinuumBackend.process_circuits`, and results retrieved using `QuantinuumBackend.get_results`. Results are received in a `BackendResult` object and represented as a series of "shots", a binary string corresponding to the measurement results for each `Qubit` after each execution of the circuit on the device. This set of shots gives a probability distribution over the set of basis states spanned by the device qubits. Broadly, the closer the probability distribution represented by the set of shots matches the ideal probability distribution, the more accurate the computation can be considered(note we are considering noise here but the nuber of shots taken also effects this). <br>

One source of error in computation is from "leakage". During execution of hardware level gates, there is a small probability for the qubit to experience leakage to electronic states outside the qubit subspace. For more information on such errors we refer to [Eliminating Leakage Errors in Hyperfine Qubits](https://arxiv.org/abs/1912.13131) by D. Hayes, D. Stack, B. Bjork, A. C. Potter, C. H. Baldwin and R. P. Stutz. <br>

Using a special circuit "gadget" and an ancilla Qubit, it's possible to detect leakage errors at the quantum circuit level and so discard erroneous shots. In this notebook we will show how to automatically run experiments with leakage detection added using `QuantinuumBackend.process_circuits` and how to process `BackendResult` returned from `QuantinuumBackend` to remove leaky results. We will also show how this scheme can improve results using a **Karl please fill in here, but a benchmarking experiment of some sort**. <br>

The leakage detection circuit, or "gadget", looks like so: <br> 

In [5]:
from pytket.extensions.quantinuum.backends.leakage_gadget import (
    get_leakage_gadget_circuit,
)
from pytket.circuit.display import render_circuit_jupyter
from pytket import Qubit, Bit

render_circuit_jupyter(
    get_leakage_gadget_circuit(
        Qubit("experiment_qubit", 0),
        Qubit("leakage_detection_qubit", 0),
        Bit("leakage_detection_bit", 0),
    )
)

The action of this Circuit is such that if leakage hasn't occurred, then the `experiment_qubit` state is unchanged and the `leakage_detection_qubit` is in state "0", while if leakage has occurred then the `leakage_detection_qubit` will be in state "1" and so can easily be detected. <br>

The `pytket-quantinuum` package has methods for automatically modifying a `pytket` `Circuit` to add leakage detection gadgets just before any end of circuit measurement gates. <br>

Let's create a basic bell pair circuit and then modify it.

In [8]:
from pytket.extensions.quantinuum.backends.leakage_gadget import get_detection_circuit
from pytket import Circuit

bell_pair_circuit = Circuit(2).H(0).CX(0, 1).measure_all()
bell_pair_leakage_detection_circuit = get_detection_circuit(
    circuit=bell_pair_circuit, n_device_qubits=4
)

render_circuit_jupyter(bell_pair_leakage_detection_circuit)

We can see that for each of the Bell pair qubits a leakage detection gadget has been appended before the final measurement. <br> <br>
The parameter `n_device_qubits` tells `get_detection_circuit` how many qubits the device has. In this case we stated there were 4 device qubits while the Bell pair circuit had two qubits, meaning a separate qubit was used for each leakage detection gadget. However, if there are too few device qubits then `get_detection_circuit` will reuse ancilla qubits to do multiple leakage detections, assigning the results to different `Bit`. We can see this by setting `n_device_qubits = 3`.

In [9]:
render_circuit_jupyter(
    get_detection_circuit(circuit=bell_pair_circuit, n_device_qubits=3)
)

The `QuantinuumBackend.process_circuits` method takes an optional `kwarg` `leakage_detection` which when `True`, will pass every circuit through `get_detection_circuit` before passing it to the hardware. This means all circuits will be executed with leakage detection added. <br>

When passing a `ResultHandle` to `QuantinuumBackend.get_result`,  the retrieved `BackendResult` will include measurement results from the leakage detection qubits. The function `prune_shots_detected_as_leaky` can be run on this

In [19]:
from pytket.extensions.quantinuum import QuantinuumBackend

circuit = Circuit(2, 2).H(0).CX(0, 1).measure_all()
# Note for demonstration purposes this is not calling the real hardware.
# Please update the device_name to reflect the hardware you want to run on.
backend = QuantinuumBackend(device_name="H1-1SC")
handle = backend.process_circuit(circuit, n_shots=100, leakage_detection=True)
result = backend.get_result(handle)

For the syntax checking Backend we've used this only returns "0000" shots, but we can see that additional `Bit` for leakage detection have been added.

In [20]:
print(result.get_counts())
print(result.c_bits)

Counter({(0, 0, 0, 0): 100})
{leakage_detection_bit[1]: 0, leakage_detection_bit[0]: 1, c[1]: 2, c[0]: 3}


We can use the method `prune_shots_detected_as_leaky` to modify this `BackendResult` object. Note that in this case there are no shots to remove as no experiment has been run, but the `Bit` corresponding with leakage detection will be removed from the returned `BackendResult`.

In [21]:
from pytket.extensions.quantinuum import prune_shots_detected_as_leaky

pruned_result = prune_shots_detected_as_leaky(result)
print(pruned_result.get_counts())
print(pruned_result.c_bits)

Counter({(0, 0): 100})
{c[1]: 0, c[0]: 1}
